In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash import html

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

education = pd.read_csv("../Data/states_all.csv").assign(
    expenditure_per_student = lambda x: x["TOTAL_EXPENDITURE"] / x["GRADES_ALL_G"],
    above_avg_math8 = lambda x: np.where(x["AVG_MATH_8_SCORE"] > 278.6,'Above Avg', 'Below Avg')
)

education.tail()

### Labels & Values

In [ ]:
dcc.Dropdown(
        id="X Column Picker",
        options=dict(zip(
            education.select_dtypes(include='number').columns,
            [column.title() for column in education.select_dtypes(include='number').columns],
        )),
        value="expenditure_per_student",
),

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2(id="header"),
    dcc.Dropdown(
        id="X Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="TOTAL_EXPENDITURE"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("header", "children"),
    Output("graph", "figure"),
    Input("X Column Picker", "value")
)

def plot_rev_scatter(x):
    if not x:
        raise PreventUpdate
    fig = px.scatter(
        education, 
        x=x, 
        y="AVG_MATH_8_SCORE",
        trendline="ols",
    )
    header = f"Math Score vs {x.title()}"
    return header, fig

if __name__ == "__main__":
    app.run_server(mode="inline", port=9271, debug=True)

### Multi Select Dropdowns

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
   dcc.Dropdown(
        id="filter",
        options=education["STATE"].unique(),
        value=["CALIFORNIA", "OREGON"],
        multi=True
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("filter", "value"),
)

def plot_rev_scatter(state):
    fig = px.line(
        (education
         .query("STATE in @state and 1992 < YEAR < 2016")
         .reset_index()
        ), 
        x="YEAR", 
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title = f"Expenditure Over Time, Selected States",
    ) 
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline", port=2023, debug=True)

In [ ]:
app = JupyterDash(__name__)

app.layout= html.Div([
    dcc.Dropdown(
        id="filter",
        options=[{
            "label": html.Span(["Serene Republic of California"], style={"color": "powderblue", "font-size": 24}),
            "value":"CALIFORNIA"
        },
        {
            "label": html.Span(["Texas Zone of Anarchy"], style={"color": "brown", "font-size": 16}),
            "value": "TEXAS"
            
        }],
        value=["CALIFORNIA"],
        multi=True
    ),
    dcc.Graph(id="graph")
])
    
@app.callback(
    Output("graph", "figure"),
    Input("filter", "value")
)

def exp_line_chart(state):
    fig = px.line(
        (education
         .query("State in @state and 1992 < YEAR < 2016")
         .groupby(["YEAR", "STATE"])
         .sum()
         .reset_index()
        ),
        x="YEAR",
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title = f"Expenditure Over Time, Selected States",
    )
    return fig

if __name__ == "main":
    app.run_server(mode="inline", debug=True, port=2048)
    


### Checklists

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="scores",
        options=[
            "AVG_MATH_4_SCORE",
            "AVG_MATH_8_SCORE",
            "AVG_READING_4_SCORE",
            "AVG_READING_8_SCORE"
        ],
        value=["AVG_MATH_4_SCORE"]
    ),
    dcc.Graph(id="graph")    
])

@app.callback(
    Output("graph", "figure"),
    Input("scores", "value")
)

def plot_hist(scores):
    fig = px.histogram(
        education,
        x=scores,
        barmode="overlay",
        title="Score Distribution"
    ).update_traces(opacity=.6)
    
    return fig

app.run_server(mode="inline", port=2084)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="filter",
        options = ["CALIFORNIA", "TEXAS", "FLORIDA", "OREGON"],
#         options= [{"label": state.title(), "value": state} for state in education["STATE"].unique()] #note this is too many options
        value=["CALIFORNIA", "TEXAS"]
    ),
    dcc.Graph(id="graph")    
])



@app.callback(
    Output("graph", "figure"),
    Input("filter", "value")
)


def exp_by_state(state):
    fig = px.line(
        (education
         .query("STATE in @state and 1992 < YEAR < 2016 ")
         .groupby(["YEAR", "STATE"])
         .sum()
         .reset_index()
        ),
        x="YEAR",
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title="Expenditure Over Time, Selected States"
    )

    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", port=4829)

### RadioItems

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id="OLS Toggle",
        options=["Trendline On", "Trendline Off"],
        value="Trendline Off"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("OLS Toggle", "value"),
)

def plot_rev_scatter(ols):
    fig = px.scatter(
        education, 
        x="expenditure_per_student", 
        y="AVG_MATH_8_SCORE",
        trendline=None if ols == "Trendline Off" else "ols",
        title = "The Relationship Between Spending and Test Scores",
        )
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=8274)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id="aggfunc",
        options=["mean", "sum", "median"],
        value="mean"
    ),
    dcc.Graph(id="graph")
])

@app.callback(
    Output("graph", "figure"),
    Input("aggfunc", "value")
)
def plot_rev_metric(metric):
    fig = px.bar(
        (education
        .query("STATE in ['CALIFORNIA', 'OREGON', 'WASHINGTON', 'IDAHO'] and 1992 < YEAR < 2016")
        .groupby(["STATE"])
        .agg({"TOTAL_EXPENDITURE": metric})
        .reset_index()
        ),
        x="STATE",
        y="TOTAL_EXPENDITURE",
        text_auto=True,
        title = f" {metric.title()} Expenditure By State"
    )
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=8274)

### Sliders

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Slider(id="spend", min=0, max=30, step=5, value=15),
    dcc.Graph(id="graph"),
])

@app.callback(Output('graph', 'figure'), Input("spend", "value"))

def bar_chart(spending):
    fig = px.bar(
        (education
         .query("expenditure_per_student > @spending")
         .groupby("YEAR", as_index=False)
         .count()),
        x="YEAR",
        y="expenditure_per_student",
        title=f"States that spent over ${spending}K per student"      
    )
    fig.update_yaxes(title="COUNT", range=[0, 50])
    fig.update_xaxes(range=[1991, 2017])
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=2323)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.RangeSlider(id="spend", min=0, max=30, step=5, value=[10, 15]),
    dcc.Graph(id="graph"),
])

@app.callback(Output('graph', 'figure'), Input("spend", "value"))

def bar_chart(spending):
    fig = px.bar(
        (education
         .query("@spending[0] <= expenditure_per_student <= @spending[1]")
         .groupby("YEAR", as_index=False)
         .count()),
        x="YEAR",
        y="expenditure_per_student",
        title=f"States that spent between ${spending[0]}-{spending[1]}K per student"      
    )
    fig.update_yaxes(title="COUNT", range=[0, 50])
    fig.update_xaxes(range=[1991, 2017])
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=2034)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
   dcc.RangeSlider(
        id="score_slider",
        min=education.query("YEAR == 2013")["AVG_MATH_8_SCORE"].min(),
        max=education.query("YEAR == 2013")["AVG_MATH_8_SCORE"].max(),
        step=5,
        value=[270, 280]
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("score_slider", "value"),
)

def plot_rev_scatter(score):
    df = (
        education
        .query("YEAR == 2013")
        .assign(
            score_cutoff =  lambda x: np.where(
                x["AVG_MATH_8_SCORE"].between(score[0], score[1]),
                'In Range', 
                'Out of Range'
            )
        )
    )
    fig = px.scatter(
        df,
        x="AVG_MATH_8_SCORE", 
        y="expenditure_per_student",
        color="score_cutoff",
        hover_name="STATE",
        title = f"Score and Expenditure vs. Threshold",
        )
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline",debug=True, port=2934)

### Date Pickers

In [ ]:
collisions = (
    pd.read_csv("../Data/NYC_Collisions.csv")
    .astype({"ACCIDENT_DATE": "datetime64"})
    .loc[:,["ACCIDENT_DATE", "BOROUGH", "COLLISION_ID"]]
    .groupby(["ACCIDENT_DATE", "BOROUGH"], as_index=False)
    .count()
    .rename(columns={
        "COLLISION_ID":"COLLISIONS", 
        "ACCIDENT_DATE":"DATE"})  
)

collisions.head()

In [ ]:
from datetime import datetime
from datetime import date

app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.DatePickerRange(
        id="date picker",
        min_date_allowed=collisions["DATE"].min(),
        max_date_allowed=collisions["DATE"].max(),
        initial_visible_month=collisions["DATE"].max(),
        start_date=collisions["DATE"].min(),
        end_date=date(2022,1,1),
#         display_format="YY-MMM-Do"
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output("graph", "figure"), 
              [Input("date picker", "start_date"),
               Input("date picker", "end_date"),])
def plot_collisions_bar(start_date, end_date):
    fig = px.bar(
        (collisions
         .loc[collisions["DATE"].between(start_date, end_date)]
         .groupby("BOROUGH", as_index=False)
         .sum()
        )
        ,
        x="COLLISIONS",
        y="BOROUGH",
        title=f"Traffic Accidents in NYC between {start_date[:10]} and {end_date[:10]}"
    )
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", port=8001, width="60%")

### Multiple Input Callbacks

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="X Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="expenditure_per_student"
),
    dcc.Dropdown(
        id="Y Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="TOTAL_EXPENDITURE"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("X Column Picker", "value"),
    Input("Y Column Picker", "value")
)

def plot_rev_scatter(x, y):
    if not x:
        raise PreventUpdate
    if not y:
        raise PreventUpdate
    fig = px.scatter(
        education, 
        x=x, 
        y=y,
        trendline="ols",
#         marginal_x="box",
#         marginal_y="box",
#         color="STATE", # can be continous
#         size="YEAR", # can't be missing
        title = "State Vs. Federal Revenue",
        )
    #fig.update_traces(marker_color=cstolor)
    return fig


if __name__ == "__main__":
    app.run_server(mode="inline", 
                   height=600, 
                   width="80%", 
                   debug=True,
                   port=6165
                  )

In [ ]:
from datetime import datetime
from datetime import date

app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.DatePickerRange(
        id="date picker",
        min_date_allowed=collisions["DATE"].min(),
        max_date_allowed=collisions["DATE"].max(),
        initial_visible_month=collisions["DATE"].max(),
        start_date=collisions["DATE"].min(),
        end_date=date(2022,1,1),
#         display_format="YY-MMM-Do"
),
    dcc.RadioItems(
        id="metric radio",
        options=["sum", "mean", "median", "min", "max"],
        value="sum"
),
    dcc.Dropdown(
        id="color dropdown",
        options=["red", "green", "blue"],
        value="blue"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("date picker", "start_date"), Input("date picker", "end_date")],
    Input("metric radio", "value"),
    Input("color dropdown", "value"),
)
def plot_collisions_bar(start_date, end_date, metric, color): 
    
    df = collisions.loc[collisions["DATE"].between(start_date, end_date)]
    
    fig = px.bar(
        (df
         .groupby("BOROUGH", as_index=False)
         .agg({"COLLISIONS": metric})
        )
        ,
        x="COLLISIONS",
        y="BOROUGH",
        title=f"Traffic Accidents in NYC between {start_date[:10]} and {end_date[:10]}"
    ).update_traces(marker_color=color)
    
    
    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", port=8001)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2(id="header"),
    dcc.Dropdown(
        id="X Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="expenditure_per_student"
),
    dcc.Dropdown(
        id="Y Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="TOTAL_EXPENDITURE"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('header', 'children'),
    Output('graph', 'figure'),
    Input("X Column Picker", "value"),
    Input("Y Column Picker", "value")
)

def plot_rev_scatter(x, y):
    if not x:
        raise PreventUpdate
    if not y:
        raise PreventUpdate
    fig = px.scatter(
        education, 
        x=x, 
        y=y,
        trendline="ols",
    )
    
    header = f"{x.title().replace('_', ' ')} Vs. {y.title().replace('_', ' ')}"
    return header, fig

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True, port=2039)

In [ ]:
from datetime import datetime
from datetime import date

app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.DatePickerRange(
        id="date picker",
        min_date_allowed=collisions["DATE"].min(),
        max_date_allowed=collisions["DATE"].max(),
        initial_visible_month=collisions["DATE"].max(),
        start_date=collisions["DATE"].min(),
        end_date=date(2022,1,1),
#         display_format="YY-MMM-Do"
),
    dcc.RadioItems(
        id="metric radio",
        options=["sum", "mean", "median", "min", "max"],
        value="sum"
),
    dcc.Dropdown(
        id="color dropdown",
        options=["red", "green", "blue"],
        value="blue"
),
    dcc.Graph(id="graph"),
    dcc.Graph(id="graph2")
])

@app.callback(
    Output("graph", "figure"), 
    Output("graph2", "figure"), 
    [Input("date picker", "start_date"), Input("date picker", "end_date")],
    Input("metric radio", "value"),
    Input("color dropdown", "value"),
)
def plot_collisions_bar(start_date, end_date, metric, color): 
    
    df = collisions.loc[collisions["DATE"].between(start_date, end_date)]
    
    fig = px.bar(
        (df
         .groupby("BOROUGH", as_index=False)
         .agg({"COLLISIONS": metric})
        )
        ,
        x="COLLISIONS",
        y="BOROUGH",
        title=f"Traffic Accidents in NYC between {start_date[:10]} and {end_date[:10]}"
    ).update_traces(marker_color=color)
    
    fig2 = px.line(
        df,
        x="DATE",
        y="COLLISIONS",
        color="BOROUGH"
    )
    
    return fig , fig2

if __name__ == "__main__":
    app.run_server(mode="inline", port=8001)